In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install required packages
!pip install fvcore tqdm matplotlib

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=f5f8bc6c09941042c58cd0c09a87a14758cff26cdc8ff5a578d00e9b1f6fed1d
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31528 sha256=6095fd3a929c81a171190044554f887e620bcb9f16bbd3be5300aaf17c45cf6c
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


In [ ]:
!git clone https://github.com/xpuria/MLDL2024_project.git

Cloning into 'MLDL2024_project'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 145 (delta 48), reused 52 (delta 24), pack-reused 57 (from 1)
Receiving objects: 100% (145/145), 46.39 KiB | 7.73 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [ ]:
# Create project directories
!mkdir -p datasets/cityscapes datasets/gta5 checkpoints plots

# First, let's extract the datasets
import os
import zipfile
from pathlib import Path

# Extract Cityscapes
print("Extracting Cityscapes...")
cityscapes_zip = '/content/drive/MyDrive/Cityscapes.zip'
cityscapes_dir = '/content/datasets/cityscapes'

with zipfile.ZipFile(cityscapes_zip, 'r') as zip_ref:
    zip_ref.extractall(cityscapes_dir)

# Extract GTA5
print("Extracting GTA5...")
gta5_zip = '/content/drive/MyDrive/GTA5.zip'
gta5_dir = '/content/datasets/gta5'

with zipfile.ZipFile(gta5_zip, 'r') as zip_ref:
    zip_ref.extractall(gta5_dir)

# Verify the structure
def print_directory_structure(path, level=0):
    indent = '  ' * level
    print(f"{indent}{os.path.basename(path)}/")
    if level < 2:  # Limit depth to avoid too much output
        for item in os.listdir(path):
            item_path = os.path.join(path, item)
            if os.path.isdir(item_path):
                print_directory_structure(item_path, level + 1)

print("\nDataset structure:")
print_directory_structure('/content/datasets')

# Print dataset statistics
print("\nDataset statistics:")
print("Cityscapes:")
print(f"Total files: {len(list(Path(cityscapes_dir).rglob('*.*')))}")
print("\nGTA5:")
print(f"Total files: {len(list(Path(gta5_dir).rglob('*.*')))}")

Extracting Cityscapes...
Extracting GTA5...

Dataset structure:
datasets/
  gta5/
    GTA5/
  cityscapes/
    Cityscapes/

Dataset statistics:
Cityscapes:
Total files: 6216

GTA5:
Total files: 5000


In [ ]:
# First, uninstall existing sympy and reinstall the correct version
!pip uninstall -y sympy
!pip install sympy==1.12

Found existing installation: sympy 1.13.1
Uninstalling sympy-1.13.1:
  Successfully uninstalled sympy-1.13.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 110.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu121 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.12 which is incompatible.


In [ ]:
import torch
from torch.utils.data import DataLoader
import numpy as np
from MLDL2024_project.datasets.cityscapes import CityscapesDataset
from MLDL2024_project.datasets.gta5 import GTA5Dataset
from MLDL2024_project.models.bisenet.build_bisenet import BiSeNet

In [ ]:
from MLDL2024_project.train import train_model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [ ]:
GTA5_train_dataset = GTA5Dataset(
        root_dir='/content/datasets/gta5/GTA5',
        height=720,
        width=1280
    )
GTA5_train_dataloader = DataLoader(GTA5_train_dataset, batch_size=2, shuffle=True, num_workers=4)

cityscapes_test_dataset = CityscapesDataset(
        root_path='/content/datasets/cityscapes/Cityscapes/Cityspaces',
        split='val',
        img_size=(512, 1024)
    )
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, batch_size=2, shuffle=True, num_workers=4)


Cityscapes val Dataset:
Number of samples: 500
Image size: (512, 1024)
Augmentations: False
Cities (3): frankfurt, lindau, munster


In [ ]:
BiSeNet_model = BiSeNet(num_classes=19, context_path='resnet18').to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 221MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 223MB/s]


In [ ]:
BiSeNet_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
BiSeNet_optimizer = torch.optim.SGD(BiSeNet_model.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)

In [ ]:
train_miou, val_miou, best_epoch = train_model(
    model=BiSeNet_model,
    criterion=BiSeNet_criterion,
    optimizer=BiSeNet_optimizer,
    train_loader=GTA5_train_dataloader,
    val_loader=cityscapes_test_dataloader,
    device=device,
    num_epochs=50,
)

print(f"Training complete. Best mIoU: {max(val_miou):.4f} at epoch {best_epoch+1}.")

Epoch 1/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.47it/s, Loss=1.0033]



Epoch 1/50:
Train Loss: 0.5914, mIoU: 22.08%

Train IoU per class:
road                : 85.86%
sidewalk            : 38.63%
building            : 53.34%
wall                : 15.26%
fence               : 0.14%
pole                : 7.31%
traffic light       : 0.03%
traffic sign        : 0.00%
vegetation          : 55.08%
terrain             : 32.00%
sky                 : 85.96%
person              : 0.00%
rider               : 0.00%
car                 : 42.38%
truck               : 3.52%
bus                 : 0.02%
train               : 0.00%
motorcycle          : 0.00%
bicycle             : 0.00%

Val Loss: 1.1108, mIoU: 16.86%

Val IoU per class:
road                : 65.87%
sidewalk            : 0.45%
building            : 62.83%
wall                : 9.40%
fence               : 0.01%
pole                : 8.62%
traffic light       : 0.00%
traffic sign        : 0.00%
vegetation          : 69.53%
terrain             : 6.81%
sky                 : 54.00%
person              : 0.00%


Epoch 2/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.58it/s, Loss=1.6931]



Epoch 2/50:
Train Loss: 0.4500, mIoU: 27.66%

Train IoU per class:
road                : 88.41%
sidewalk            : 53.06%
building            : 61.31%
wall                : 27.14%
fence               : 1.29%
pole                : 18.69%
traffic light       : 0.00%
traffic sign        : 0.00%
vegetation          : 63.71%
terrain             : 42.97%
sky                 : 90.23%
person              : 0.00%
rider               : 0.00%
car                 : 56.21%
truck               : 22.21%
bus                 : 0.00%
train               : 0.39%
motorcycle          : 0.00%
bicycle             : 0.00%

Val Loss: 1.2402, mIoU: 15.29%

Val IoU per class:
road                : 79.10%
sidewalk            : 0.13%
building            : 56.74%
wall                : 8.66%
fence               : 0.00%
pole                : 0.54%
traffic light       : 0.00%
traffic sign        : 0.00%
vegetation          : 52.91%
terrain             : 7.75%
sky                 : 46.48%
person              : 0.00

Epoch 3/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.48it/s, Loss=1.3961]


...


Epoch 48/50:
Train Loss: 0.1240, mIoU: 75.04%

Train IoU per class:
road                : 96.13%
sidewalk            : 85.35%
building            : 87.07%
wall                : 79.01%
fence               : 64.24%
pole                : 52.33%
traffic light       : 52.36%
traffic sign        : 61.93%
vegetation          : 81.82%
terrain             : 78.66%
sky                 : 95.32%
person              : 52.01%
rider               : 55.04%
car                 : 91.20%
truck               : 91.02%
bus                 : 94.17%
train               : 92.34%
motorcycle          : 70.01%
bicycle             : 45.76%

Val Loss: 1.7290, mIoU: 18.69%

Val IoU per class:
road                : 38.20%
sidewalk            : 8.32%
building            : 63.68%
wall                : 3.13%
fence               : 5.02%
pole                : 17.16%
traffic light       : 12.35%
traffic sign        : 3.31%
vegetation          : 59.95%
terrain             : 3.94%
sky                 : 64.51%
person        

Epoch 49/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.41it/s, Loss=1.0273]



Epoch 49/50:
Train Loss: 0.1233, mIoU: 74.92%

Train IoU per class:
road                : 96.16%
sidewalk            : 85.45%
building            : 87.16%
wall                : 79.08%
fence               : 64.37%
pole                : 52.47%
traffic light       : 52.27%
traffic sign        : 62.18%
vegetation          : 81.83%
terrain             : 78.63%
sky                 : 95.32%
person              : 51.87%
rider               : 53.91%
car                 : 91.27%
truck               : 91.11%
bus                 : 94.50%
train               : 91.53%
motorcycle          : 70.05%
bicycle             : 44.36%

Val Loss: 1.8766, mIoU: 18.19%

Val IoU per class:
road                : 35.95%
sidewalk            : 8.18%
building            : 64.11%
wall                : 3.63%
fence               : 5.79%
pole                : 17.31%
traffic light       : 11.08%
traffic sign        : 3.05%
vegetation          : 54.35%
terrain             : 3.31%
sky                 : 62.74%
person        

Epoch 50/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.44it/s, Loss=1.0022]



Epoch 50/50:
Train Loss: 0.1231, mIoU: 75.10%

Train IoU per class:
road                : 96.17%
sidewalk            : 85.51%
building            : 87.14%
wall                : 79.27%
fence               : 64.42%
pole                : 52.43%
traffic light       : 52.27%
traffic sign        : 61.90%
vegetation          : 81.86%
terrain             : 78.72%
sky                 : 95.32%
person              : 51.80%
rider               : 55.24%
car                 : 91.25%
truck               : 91.17%
bus                 : 94.56%
train               : 92.20%
motorcycle          : 70.30%
bicycle             : 45.39%

Val Loss: 1.6614, mIoU: 19.29%

Val IoU per class:
road                : 44.31%
sidewalk            : 8.18%
building            : 65.68%
wall                : 3.80%
fence               : 6.00%
pole                : 16.72%
traffic light       : 12.23%
traffic sign        : 3.22%
vegetation          : 56.55%
terrain             : 3.82%
sky                 : 62.64%
person        

In [ ]:
from torchvision import transforms

In [ ]:
augment_1 = transforms.RandomInvert(p=0.5)

In [ ]:
GTA5_train_dataset = GTA5Dataset(
        root_dir='/content/datasets/gta5/GTA5',
        height=720,
        width=1280,
        augment = augment_1
    )
GTA5_train_dataloader_augmented_1 = DataLoader(GTA5_train_dataset, batch_size=2, shuffle=True, num_workers=4)

cityscapes_test_dataset = CityscapesDataset(
        root_path='/content/datasets/cityscapes/Cityscapes/Cityspaces',
        split='val',
        img_size=(512, 1024)
    )
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, batch_size=2, shuffle=True, num_workers=4)


Cityscapes val Dataset:
Number of samples: 500
Image size: (512, 1024)
Augmentations: False
Cities (3): frankfurt, lindau, munster


In [ ]:
BiSeNet_model = BiSeNet(num_classes=19, context_path='resnet18').to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 220MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 209MB/s]


In [ ]:
BiSeNet_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
BiSeNet_optimizer = torch.optim.SGD(BiSeNet_model.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)

In [ ]:
train_miou, val_miou, best_epoch = train_model(
    model=BiSeNet_model,
    criterion=BiSeNet_criterion,
    optimizer=BiSeNet_optimizer,
    train_loader=GTA5_train_dataloader_augmented_1,
    val_loader=cityscapes_test_dataloader,
    device=device,
    num_epochs=50,
)

print(f"Training complete. Best mIoU: {max(val_miou):.4f} at epoch {best_epoch+1}.")

Epoch 1/50 (Val): 100%|██████████| 250/250 [00:15<00:00, 15.63it/s, Loss=1.6111]



Epoch 1/50:
Train Loss: 0.7186, mIoU: 18.07%

Train IoU per class:
road                : 83.30%
sidewalk            : 29.57%
building            : 48.05%
wall                : 6.95%
fence               : 0.01%
pole                : 3.00%
traffic light       : 0.03%
traffic sign        : 0.00%
vegetation          : 44.29%
terrain             : 18.11%
sky                 : 77.31%
person              : 0.00%
rider               : 0.00%
car                 : 30.77%
truck               : 1.88%
bus                 : 0.00%
train               : 0.00%
motorcycle          : 0.03%
bicycle             : 0.00%

Val Loss: 1.7423, mIoU: 12.25%

Val IoU per class:
road                : 25.12%
sidewalk            : 0.41%
building            : 63.73%
wall                : 3.06%
fence               : 0.00%
pole                : 0.94%
traffic light       : 0.00%
traffic sign        : 0.00%
vegetation          : 62.77%
terrain             : 6.66%
sky                 : 54.46%
person              : 0.00%
r

Epoch 2/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.50it/s, Loss=1.2478]



Epoch 2/50:
Train Loss: 0.5497, mIoU: 23.07%

Train IoU per class:
road                : 86.58%
sidewalk            : 44.08%
building            : 55.08%
wall                : 17.87%
fence               : 0.01%
pole                : 11.60%
traffic light       : 0.00%
traffic sign        : 0.00%
vegetation          : 54.04%
terrain             : 32.64%
sky                 : 84.63%
person              : 0.00%
rider               : 0.00%
car                 : 45.13%
truck               : 6.43%
bus                 : 0.03%
train               : 0.20%
motorcycle          : 0.00%
bicycle             : 0.00%

Val Loss: 1.5465, mIoU: 13.29%

Val IoU per class:
road                : 26.20%
sidewalk            : 6.85%
building            : 60.23%
wall                : 9.34%
fence               : 0.02%
pole                : 4.70%
traffic light       : 0.00%
traffic sign        : 0.00%
vegetation          : 64.00%
terrain             : 11.85%
sky                 : 49.48%
person              : 0.00

Epoch 3/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.43it/s, Loss=1.9459]


...


Epoch 48/50:
Train Loss: 0.1377, mIoU: 70.92%

Train IoU per class:
road                : 95.78%
sidewalk            : 83.31%
building            : 85.94%
wall                : 76.75%
fence               : 61.81%
pole                : 49.30%
traffic light       : 45.65%
traffic sign        : 57.43%
vegetation          : 80.04%
terrain             : 76.53%
sky                 : 94.87%
person              : 48.42%
rider               : 49.53%
car                 : 89.78%
truck               : 89.66%
bus                 : 92.92%
train               : 91.84%
motorcycle          : 62.79%
bicycle             : 15.08%

Val Loss: 1.3482, mIoU: 20.52%

Val IoU per class:
road                : 43.02%
sidewalk            : 17.14%
building            : 66.66%
wall                : 4.42%
fence               : 5.85%
pole                : 14.65%
traffic light       : 10.45%
traffic sign        : 4.10%
vegetation          : 74.89%
terrain             : 7.48%
sky                 : 65.46%
person       

Epoch 49/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.29it/s, Loss=1.3903]



Epoch 49/50:
Train Loss: 0.1377, mIoU: 70.86%

Train IoU per class:
road                : 95.77%
sidewalk            : 83.42%
building            : 85.96%
wall                : 76.60%
fence               : 61.85%
pole                : 49.31%
traffic light       : 45.60%
traffic sign        : 57.39%
vegetation          : 80.08%
terrain             : 76.48%
sky                 : 94.87%
person              : 48.30%
rider               : 50.94%
car                 : 89.67%
truck               : 89.71%
bus                 : 93.52%
train               : 91.42%
motorcycle          : 62.82%
bicycle             : 12.69%

Val Loss: 1.3582, mIoU: 20.09%

Val IoU per class:
road                : 43.16%
sidewalk            : 15.54%
building            : 65.81%
wall                : 3.92%
fence               : 5.31%
pole                : 13.47%
traffic light       : 9.27%
traffic sign        : 3.95%
vegetation          : 74.11%
terrain             : 8.00%
sky                 : 65.31%
person        

Epoch 50/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.40it/s, Loss=1.0986]



Epoch 50/50:
Train Loss: 0.1368, mIoU: 71.10%

Train IoU per class:
road                : 95.81%
sidewalk            : 83.42%
building            : 86.04%
wall                : 76.90%
fence               : 61.88%
pole                : 49.46%
traffic light       : 45.42%
traffic sign        : 57.96%
vegetation          : 80.14%
terrain             : 76.59%
sky                 : 94.90%
person              : 48.29%
rider               : 50.21%
car                 : 89.76%
truck               : 89.61%
bus                 : 93.23%
train               : 91.76%
motorcycle          : 63.56%
bicycle             : 16.01%

Val Loss: 1.3465, mIoU: 20.49%

Val IoU per class:
road                : 44.12%
sidewalk            : 16.07%
building            : 67.95%
wall                : 4.44%
fence               : 6.10%
pole                : 14.59%
traffic light       : 10.01%
traffic sign        : 3.88%
vegetation          : 73.86%
terrain             : 7.36%
sky                 : 66.02%
person       

In [ ]:
train_miou

[18.06766392875651,
 23.069419249643676,
 25.166013948616428,
 27.813471381897998,
 29.941065056780502,
 31.731110763911342,
 33.429864450973675,
 34.12926159601145,
 35.803550630582535,
 38.80301519109477,
 41.22698581935549,
 42.426978525897205,
 44.41373723455251,
 47.53510771088104,
 50.39678826622672,
 51.4909926615365,
 53.96028763165,
 55.22885464706122,
 55.832622087422244,
 57.622904103252196,
 58.59916561638876,
 58.974764934577095,
 60.6560919291833,
 61.45819359578325,
 62.897443478592926,
 63.86233020148621,
 63.88827107728061,
 64.86821304515169,
 65.93855428147867,
 66.17382826959049,
 66.46455705173805,
 66.79171486804208,
 67.24044481982499,
 67.70093947888176,
 67.71421269829942,
 68.12080078156161,
 68.54177871650134,
 68.87030772911417,
 69.06656338713388,
 69.45532610745104,
 69.70650909477177,
 69.85178019061097,
 69.86412671987397,
 70.53817447484899,
 70.28090946605104,
 70.94528837416178,
 70.83218868330782,
 70.9183620638013,
 70.86430266477372,
 71.1023227474

In [ ]:
val_miou

[12.251841991008295,
 13.294484892992923,
 8.660363941126478,
 10.609672140411046,
 11.809284059283161,
 13.423876188180529,
 8.697209223185052,
 11.949538649001301,
 11.649794275529034,
 14.875970236967042,
 15.900023031105986,
 16.053643619930817,
 16.567658214778152,
 16.45392809389394,
 17.792511671717683,
 17.90853950418253,
 18.33463520854605,
 16.842640910827075,
 18.63998594150347,
 18.66115664432388,
 17.340041699843496,
 17.977413303885807,
 18.37733763788229,
 17.87888147454051,
 20.480889719616815,
 19.747345606270112,
 19.40329385164007,
 20.22822271890712,
 18.87150409158021,
 18.778875720455275,
 19.48396991466382,
 21.35609751556446,
 20.187549751839995,
 20.510726415311705,
 20.666272761534643,
 20.3669504185184,
 20.837075613790017,
 20.72960270557886,
 20.922362026344526,
 21.04280950089368,
 20.694970839569592,
 20.446054563869875,
 20.21797320577139,
 20.338135739461745,
 20.456360771616485,
 20.840547698116136,
 20.40059910281853,
 20.51733205469764,
 20.085181173

In [ ]:
best_epoch

31

In [ ]:
augment_2 = transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3)

In [ ]:
GTA5_train_dataset = GTA5Dataset(
        root_dir='/content/datasets/gta5/GTA5',
        height=720,
        width=1280,
        augment = augment_2
    )
GTA5_train_dataloader_augmented_2 = DataLoader(GTA5_train_dataset, batch_size=2, shuffle=True, num_workers=4)

cityscapes_test_dataset = CityscapesDataset(
        root_path='/content/datasets/cityscapes/Cityscapes/Cityspaces',
        split='val',
        img_size=(512, 1024)
    )
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, batch_size=2, shuffle=True, num_workers=4)


Cityscapes val Dataset:
Number of samples: 500
Image size: (512, 1024)
Augmentations: False
Cities (3): frankfurt, lindau, munster


In [ ]:
train_miou, val_miou, best_epoch = train_model(
    model=BiSeNet_model,
    criterion=BiSeNet_criterion,
    optimizer=BiSeNet_optimizer,
    train_loader=GTA5_train_dataloader_augmented_2,
    val_loader=cityscapes_test_dataloader,
    device=device,
    num_epochs=50,
)

print(f"Training complete. Best mIoU: {max(val_miou):.4f} at epoch {best_epoch+1}.")

Epoch 1/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.50it/s, Loss=1.0222]



Epoch 1/50:
Train Loss: 0.2488, mIoU: 55.13%

Train IoU per class:
road                : 92.36%
sidewalk            : 66.80%
building            : 76.09%
wall                : 55.08%
fence               : 39.18%
pole                : 42.00%
traffic light       : 33.81%
traffic sign        : 39.06%
vegetation          : 73.54%
terrain             : 58.16%
sky                 : 93.06%
person              : 35.21%
rider               : 33.71%
car                 : 80.65%
truck               : 65.60%
bus                 : 50.43%
train               : 67.94%
motorcycle          : 44.80%
bicycle             : 0.00%

Val Loss: 1.1513, mIoU: 21.58%

Val IoU per class:
road                : 47.42%
sidewalk            : 16.07%
building            : 63.32%
wall                : 3.88%
fence               : 8.85%
pole                : 16.60%
traffic light       : 9.74%
traffic sign        : 6.12%
vegetation          : 74.36%
terrain             : 10.45%
sky                 : 77.08%
person         

Epoch 2/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.53it/s, Loss=0.7701]



Epoch 2/50:
Train Loss: 0.2014, mIoU: 61.75%

Train IoU per class:
road                : 93.71%
sidewalk            : 74.08%
building            : 79.96%
wall                : 61.69%
fence               : 47.05%
pole                : 43.92%
traffic light       : 37.46%
traffic sign        : 43.40%
vegetation          : 76.16%
terrain             : 64.73%
sky                 : 94.01%
person              : 40.78%
rider               : 37.04%
car                 : 84.90%
truck               : 80.61%
bus                 : 81.84%
train               : 80.14%
motorcycle          : 50.72%
bicycle             : 0.97%

Val Loss: 1.2677, mIoU: 23.25%

Val IoU per class:
road                : 43.85%
sidewalk            : 17.74%
building            : 71.48%
wall                : 13.06%
fence               : 8.50%
pole                : 18.51%
traffic light       : 12.32%
traffic sign        : 7.29%
vegetation          : 76.30%
terrain             : 19.79%
sky                 : 81.62%
person       

Epoch 3/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.18it/s, Loss=0.9353]


...


Epoch 48/50:
Train Loss: 0.0998, mIoU: 79.47%

Train IoU per class:
road                : 96.89%
sidewalk            : 88.21%
building            : 89.23%
wall                : 83.79%
fence               : 70.04%
pole                : 56.96%
traffic light       : 60.71%
traffic sign        : 69.84%
vegetation          : 83.58%
terrain             : 81.96%
sky                 : 95.46%
person              : 58.41%
rider               : 65.47%
car                 : 92.94%
truck               : 93.09%
bus                 : 95.92%
train               : 94.06%
motorcycle          : 75.49%
bicycle             : 57.83%

Val Loss: 1.0514, mIoU: 24.88%

Val IoU per class:
road                : 68.25%
sidewalk            : 20.23%
building            : 74.07%
wall                : 13.41%
fence               : 5.12%
pole                : 17.35%
traffic light       : 10.24%
traffic sign        : 3.27%
vegetation          : 75.28%
terrain             : 13.74%
sky                 : 80.49%
person     

Epoch 49/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.38it/s, Loss=0.6811]



Epoch 49/50:
Train Loss: 0.0997, mIoU: 79.53%

Train IoU per class:
road                : 96.90%
sidewalk            : 88.21%
building            : 89.21%
wall                : 83.85%
fence               : 70.12%
pole                : 56.96%
traffic light       : 60.82%
traffic sign        : 69.70%
vegetation          : 83.59%
terrain             : 82.06%
sky                 : 95.45%
person              : 58.51%
rider               : 66.16%
car                 : 92.99%
truck               : 93.03%
bus                 : 95.82%
train               : 94.02%
motorcycle          : 75.94%
bicycle             : 57.75%

Val Loss: 1.0564, mIoU: 24.72%

Val IoU per class:
road                : 68.20%
sidewalk            : 18.52%
building            : 73.76%
wall                : 12.54%
fence               : 5.28%
pole                : 18.78%
traffic light       : 10.65%
traffic sign        : 2.69%
vegetation          : 75.40%
terrain             : 14.09%
sky                 : 80.60%
person     

Epoch 50/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.40it/s, Loss=0.7855]



Epoch 50/50:
Train Loss: 0.0996, mIoU: 79.48%

Train IoU per class:
road                : 96.90%
sidewalk            : 88.23%
building            : 89.25%
wall                : 83.75%
fence               : 70.16%
pole                : 56.97%
traffic light       : 60.76%
traffic sign        : 70.06%
vegetation          : 83.58%
terrain             : 82.06%
sky                 : 95.46%
person              : 58.72%
rider               : 66.30%
car                 : 92.99%
truck               : 93.07%
bus                 : 95.93%
train               : 94.07%
motorcycle          : 75.87%
bicycle             : 56.07%

Val Loss: 1.0838, mIoU: 24.54%

Val IoU per class:
road                : 65.63%
sidewalk            : 20.18%
building            : 73.21%
wall                : 12.25%
fence               : 4.47%
pole                : 17.20%
traffic light       : 10.96%
traffic sign        : 3.12%
vegetation          : 76.05%
terrain             : 15.17%
sky                 : 81.34%
person     

In [ ]:
train_miou, val_miou, best_epoch

([55.13061242235571,
  61.745957268396744,
  60.319041688332376,
  65.20321640504699,
  67.16996681859047,
  69.405643600576,
  69.94469754664323,
  71.15197651166318,
  71.95178189134637,
  72.05393672256193,
  72.53648069800863,
  73.16817242824983,
  73.76331702254193,
  74.12319378232884,
  74.4820215734085,
  74.65271052409082,
  74.8191398633612,
  75.64947961084026,
  75.63278859117307,
  75.20096920255767,
  75.85438580356612,
  76.18083706636848,
  76.46337779129894,
  76.61267088660493,
  76.99833328926185,
  77.07555345599812,
  77.38678576600206,
  77.49411860384127,
  77.74123249766247,
  77.76496256896424,
  77.8864544805615,
  77.9417846451883,
  78.17308950168798,
  78.32316019735822,
  78.26433895614603,
  78.6537144807648,
  78.62299516935137,
  78.74470709621534,
  78.83139288854228,
  78.92395609502249,
  79.08544129758681,
  79.11839177490411,
  79.17964274419664,
  79.22614048553156,
  79.31443588188628,
  79.33388277722766,
  79.37705352548592,
  79.4667394672597

In [ ]:
GTA5_train_dataset = GTA5Dataset(
        root_dir='/content/datasets/gta5/GTA5',
        height=720,
        width=1280,
        augment = [augment_1, augment_2]
    )
GTA5_train_dataloader_augmented_1_2 = DataLoader(GTA5_train_dataset, batch_size=2, shuffle=True, num_workers=4)

cityscapes_test_dataset = CityscapesDataset(
        root_path='/content/datasets/cityscapes/Cityscapes/Cityspaces',
        split='val',
        img_size=(512, 1024)
    )
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, batch_size=2, shuffle=True, num_workers=4)


Cityscapes val Dataset:
Number of samples: 500
Image size: (512, 1024)
Augmentations: False
Cities (3): frankfurt, lindau, munster


In [ ]:
train_miou, val_miou, best_epoch = train_model(
    model=BiSeNet_model,
    criterion=BiSeNet_criterion,
    optimizer=BiSeNet_optimizer,
    train_loader=GTA5_train_dataloader_augmented_1_2,
    val_loader=cityscapes_test_dataloader,
    device=device,
    num_epochs=50,
)

print(f"Training complete. Best mIoU: {max(val_miou):.4f} at epoch {best_epoch+1}.")

Epoch 1/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.47it/s, Loss=1.2567]



Epoch 1/50:
Train Loss: 0.7468, mIoU: 16.87%

Train IoU per class:
road                : 82.80%
sidewalk            : 29.28%
building            : 45.11%
wall                : 4.64%
fence               : 0.00%
pole                : 0.78%
traffic light       : 0.02%
traffic sign        : 0.06%
vegetation          : 42.21%
terrain             : 16.77%
sky                 : 74.33%
person              : 0.03%
rider               : 0.00%
car                 : 22.77%
truck               : 1.21%
bus                 : 0.39%
train               : 0.00%
motorcycle          : 0.00%
bicycle             : 0.06%

Val Loss: 1.2003, mIoU: 16.89%

Val IoU per class:
road                : 76.34%
sidewalk            : 3.49%
building            : 64.19%
wall                : 3.17%
fence               : 0.00%
pole                : 0.08%
traffic light       : 0.00%
traffic sign        : 0.00%
vegetation          : 54.96%
terrain             : 6.01%
sky                 : 70.57%
person              : 0.00%
r

Epoch 2/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.52it/s, Loss=1.5569]



Epoch 2/50:
Train Loss: 0.5587, mIoU: 22.76%

Train IoU per class:
road                : 86.46%
sidewalk            : 41.75%
building            : 55.09%
wall                : 16.33%
fence               : 0.00%
pole                : 6.96%
traffic light       : 0.00%
traffic sign        : 0.00%
vegetation          : 54.82%
terrain             : 30.00%
sky                 : 82.55%
person              : 0.00%
rider               : 0.00%
car                 : 48.54%
truck               : 9.26%
bus                 : 0.51%
train               : 0.16%
motorcycle          : 0.00%
bicycle             : 0.00%

Val Loss: 1.4962, mIoU: 13.98%

Val IoU per class:
road                : 44.73%
sidewalk            : 5.89%
building            : 63.89%
wall                : 6.61%
fence               : 0.00%
pole                : 1.44%
traffic light       : 0.00%
traffic sign        : 0.00%
vegetation          : 54.30%
terrain             : 8.98%
sky                 : 56.53%
person              : 0.00%


Epoch 3/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.56it/s, Loss=1.8138]


...


Epoch 48/50:
Train Loss: 0.1487, mIoU: 70.19%

Train IoU per class:
road                : 95.48%
sidewalk            : 81.64%
building            : 84.98%
wall                : 74.68%
fence               : 60.38%
pole                : 47.74%
traffic light       : 40.68%
traffic sign        : 54.63%
vegetation          : 78.67%
terrain             : 74.60%
sky                 : 94.31%
person              : 47.14%
rider               : 53.83%
car                 : 89.09%
truck               : 88.86%
bus                 : 92.49%
train               : 88.61%
motorcycle          : 65.86%
bicycle             : 19.88%

Val Loss: 1.0677, mIoU: 24.41%

Val IoU per class:
road                : 58.47%
sidewalk            : 20.19%
building            : 74.71%
wall                : 9.71%
fence               : 3.30%
pole                : 20.56%
traffic light       : 8.47%
traffic sign        : 4.91%
vegetation          : 73.61%
terrain             : 12.24%
sky                 : 80.38%
person       

Epoch 49/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.44it/s, Loss=0.6792]



Epoch 49/50:
Train Loss: 0.1480, mIoU: 70.30%

Train IoU per class:
road                : 95.50%
sidewalk            : 81.59%
building            : 85.09%
wall                : 74.98%
fence               : 60.66%
pole                : 47.93%
traffic light       : 40.66%
traffic sign        : 55.18%
vegetation          : 78.79%
terrain             : 74.61%
sky                 : 94.35%
person              : 47.06%
rider               : 54.66%
car                 : 89.24%
truck               : 88.62%
bus                 : 91.64%
train               : 89.96%
motorcycle          : 65.63%
bicycle             : 19.63%

Val Loss: 1.0435, mIoU: 24.72%

Val IoU per class:
road                : 58.28%
sidewalk            : 20.78%
building            : 74.86%
wall                : 9.52%
fence               : 4.73%
pole                : 20.87%
traffic light       : 9.21%
traffic sign        : 6.41%
vegetation          : 74.10%
terrain             : 12.41%
sky                 : 80.34%
person       

Epoch 50/50 (Val): 100%|██████████| 250/250 [00:16<00:00, 15.48it/s, Loss=1.0302]



Epoch 50/50:
Train Loss: 0.1476, mIoU: 70.44%

Train IoU per class:
road                : 95.51%
sidewalk            : 81.75%
building            : 85.08%
wall                : 75.11%
fence               : 60.70%
pole                : 47.99%
traffic light       : 41.14%
traffic sign        : 54.90%
vegetation          : 78.74%
terrain             : 74.54%
sky                 : 94.35%
person              : 47.71%
rider               : 54.61%
car                 : 89.15%
truck               : 88.92%
bus                 : 92.19%
train               : 90.73%
motorcycle          : 65.67%
bicycle             : 19.65%

Val Loss: 0.9662, mIoU: 24.97%

Val IoU per class:
road                : 62.92%
sidewalk            : 20.84%
building            : 74.76%
wall                : 9.35%
fence               : 3.76%
pole                : 20.53%
traffic light       : 8.16%
traffic sign        : 5.74%
vegetation          : 73.59%
terrain             : 11.81%
sky                 : 80.24%
person       

In [ ]:
train_miou, val_miou, best_epoch

([16.867552136353574,
  22.75917147752999,
  25.90325545403114,
  28.16874913273609,
  30.21169280212583,
  32.837666425267834,
  34.76947333963783,
  36.26940471226235,
  37.959782151652114,
  40.88921661095709,
  42.361747038503744,
  44.39155752967383,
  46.53826486984543,
  48.63388790318795,
  49.45724406164886,
  52.625948071436376,
  53.486597539260735,
  54.439475887743626,
  56.09513136371347,
  57.60316824362528,
  59.2396620487894,
  60.40614022778603,
  61.037988856141745,
  61.80476496615125,
  62.60239390116665,
  63.06866714533719,
  63.8886754393139,
  64.1473637914701,
  64.40571589559538,
  65.32334548640132,
  65.63487669368165,
  65.75133189827484,
  66.39234722409678,
  66.52844439526412,
  66.80611666540528,
  66.56357659623885,
  67.4311590170521,
  67.8869544657219,
  67.92325776116846,
  68.42827226010021,
  69.09914123409466,
  69.01197734425378,
  69.04601393109925,
  69.389056190254,
  70.0504126149628,
  70.15133369433116,
  70.05128239775831,
  70.18659397